In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1yDbLLrmh6UQ-tu47wr1lbOpb3aF_Q-Pg'})
download.GetContentFile('gan.zip')
#https://drive.google.com/open?id=1yDbLLrmh6UQ-tu47wr1lbOpb3aF_Q-Pg

In [0]:
import zipfile
with zipfile.ZipFile("gan.zip","r") as zip_ref:
    zip_ref.extractall()

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchSize=64
imageSize=64
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) 

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Files already downloaded and verified


In [0]:
def weights_init(m):
  classname=m.__class__.__name__
  if(classname.find('Conv')!=-1):
    m.weight.data.normal_(0.0,0.02)
  elif classname.find('BatchNorm')!= -1:
    m.weight.data.normal_(1.0,0.02)
    m.bias.data.fill_(0)

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('gpu')  # you can continue going on here, like cuda:1 cuda:2....etc. 
else:
    device = torch.device("cpu")

gpu


In [0]:
class G(nn.Module):
  def __init__(self):
    super(G,self).__init__()
    self.main=nn.Sequential(
        #in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros'
        nn.ConvTranspose2d(100,512,4,1,0,bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        nn.ConvTranspose2d(512,256,4,2,1,bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.ConvTranspose2d(256,128,4,2,1,bias=True),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        nn.ConvTranspose2d(128,64,4,2,1,bias=True),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.ConvTranspose2d(64,3,4,2,1,bias=False),
        nn.Tanh()
    )
    
  def forward(self,input):
    output=self.main(input)
    return output

In [0]:
netG=G()
netG.apply(weights_init)
netG=netG.to(device)

In [14]:
netG

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [0]:
class D(nn.Module):
  def __init__(self):
    super(D,self).__init__()
    self.main=nn.Sequential(
    nn.Conv2d(3,64,4,2,1,bias=False),
    nn.LeakyReLU(negative_slope=0.2,inplace=True),
    nn.Conv2d(64,128,4,2,1,bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(negative_slope=0.2,inplace=True),
    nn.Conv2d(128,256,4,2,1,bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(negative_slope=0.2,inplace=True),
    nn.Conv2d(256,512,4,2,1,bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(negative_slope=0.2,inplace=True),
    nn.Conv2d(512,1,4,1,0,bias=False),
    nn.Sigmoid()
    )

  def forward(self,input):
    output=self.main(input)
    return output.view(-1)

In [0]:
netD=D()
netD.apply(weights_init)
netD=netD.to(device)

In [16]:
netD

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [0]:
criterion=nn.BCELoss()
optimizerD=optim.Adam(netD.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG=optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))

In [32]:
num_epochs=21
for epoch in range(num_epochs):
  for i,data in enumerate(dataloader,0):
    
    netD.zero_grad()
    real,_=data
    input=Variable(real).to(device)
    target_real = Variable(torch.ones(input.size()[0])).to(device)
    output_real=netD(input)
    errD_real=criterion(output_real,target_real)
    
    #generate random noise vector for input to generator
    noise=Variable(torch.randn(input.size()[0],100,1,1)).to(device) # like 100 feature maps of size 1x1
    fake=netG(noise)
    fake=fake.to(device)
    target_fake = Variable(torch.zeros(input.size()[0])).to(device)
    output_fake=netD(fake.detach())
    errD_fake=criterion(output_fake,target_fake)

    #backpropagate total error
    errD=errD_real+errD_fake
    errD.backward()
    optimizerD.step()

    #train the generator now
    netG.zero_grad()
    #this time generator will try to confuse the discriminator so target=1
    target_fake_gen = Variable(torch.ones(input.size()[0])).to(device)
    output_fake_gen=netD(fake)
    errG=criterion(output_fake_gen,target_fake_gen)
    errG.backward()
    optimizerG.step()
    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, num_epochs, i, len(dataloader), errD.item(), errG.item()))
    if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "./my_results", normalize = True) 
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./my_results", epoch), normalize = True) 

Streaming output truncated to the last 5000 lines.
[14/21][475/782] Loss_D: 10.4202 Loss_G: 7.4233
[14/21][476/782] Loss_D: 2.3725 Loss_G: 1.9818
[14/21][477/782] Loss_D: 0.4900 Loss_G: 0.9930
[14/21][478/782] Loss_D: 1.4376 Loss_G: 4.5642
[14/21][479/782] Loss_D: 1.0784 Loss_G: 2.2208
[14/21][480/782] Loss_D: 0.6524 Loss_G: 1.6834
[14/21][481/782] Loss_D: 0.7453 Loss_G: 3.3442
[14/21][482/782] Loss_D: 0.7194 Loss_G: 2.1031
[14/21][483/782] Loss_D: 0.8816 Loss_G: 1.6358
[14/21][484/782] Loss_D: 0.8718 Loss_G: 2.5889
[14/21][485/782] Loss_D: 0.8416 Loss_G: 1.6661
[14/21][486/782] Loss_D: 0.7955 Loss_G: 2.6872
[14/21][487/782] Loss_D: 0.5381 Loss_G: 2.2600
[14/21][488/782] Loss_D: 0.7901 Loss_G: 1.6296
[14/21][489/782] Loss_D: 0.8906 Loss_G: 2.9018
[14/21][490/782] Loss_D: 0.9165 Loss_G: 1.1274
[14/21][491/782] Loss_D: 0.8662 Loss_G: 3.7892
[14/21][492/782] Loss_D: 0.7155 Loss_G: 2.0134
[14/21][493/782] Loss_D: 0.6726 Loss_G: 2.8180
[14/21][494/782] Loss_D: 0.9019 Loss_G: 1.3684
[14/21][

In [18]:
cd "gan's"

/content/gan's


In [19]:
ls

data/  dcgan_commented.py  dcgan_nocomment.py  my_results/  results/
